<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz
! pip install "tqdm==4.43.0"

Cloning into 'StockSellRulesChecker'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 96 (delta 27), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (96/96), done.
     |████████████████████████████████| 9.5MB 5.1MB/s 
     |████████████████████████████████| 1.3MB 48.0MB/s 
     |████████████████████████████████| 143kB 34.8MB/s 
     |████████████████████████████████| 296kB 43.2MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=e8b2c79fc9a89c836f16935114b5e4729cb8e8e11549efebc54c92e3c6be0732
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=68053df538da9fd1e2fa906c1c2f55f66970c2c9293b4083f5413070e327193e
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b467

  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-6r4gkvox
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-6r4gkvox
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=81dcda108ba8f8465a130bc0dadf655488888d904cb89dcea8402245bd0d0335
  Stored in directory: /tmp/pip-ephem-wheel-cache-akson8f8/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4
     |████████████████████████████████| 61kB 2.6MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [47]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time
from tqdm.contrib.concurrent import process_map


In [48]:
eps_5year_over5pct = False
eps_qoq_over5pct = False
eps_yoy_over5pct = False
sales_5years_over5pct = False
sales_qoq_over5pct = False

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [49]:
print(len(stock_list.data))

7971


In [50]:

def moving_average(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())

def SMA200_slope_positive_rule(yahoo_df, days=21):
  for day in range(days):
    if moving_average(yahoo_df, days=200, delta=day) > moving_average(yahoo_df, days=200, delta=day+1):
      continue
    else:
      return False
  return True

def screen_stock(stock):
  if stock["Ticker"] == "":
    return
  screened_stocks[stock['Ticker']] = {}
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    return

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))/100
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))/100
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))/100
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))/100
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= .18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= .25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= .35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  SMA200_slope_rule = SMA200_slope_positive_rule(yahoo_df, days=21)
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule
  return screened_stocks

screened_stocks = {}
results = process_map(screen_stock, stock_list, max_workers=16)



    


In [51]:
screened_stocks = {}
for d in results:
  if d is not None:
    screened_stocks.update(d)
  
output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

In [52]:
primary_rules = ['eps_QoQ_yearly_rule', 'sales_QoQ_yearly_rule', 'SMA200_slope_rule', 
                 'SMA150_greater_SMA200_rule', 'SMA50_greater_SMA150_rule', 'close_greater_SMA50_rule', 
                 'week52_span_rule', 'close_above_52weekhigh_rule']

secondary_rules = ['vol_price_rule', 'inst_ownership_rule', 'shares_outstanding_rule']
df_out['Primary Passed Tests'] = (df_out[list(primary_rules)]).sum(1)
df_out['Secondary Passed Tests'] = (df_out[list(secondary_rules)]).sum(1)

cols = df_out.columns.tolist()

for rule in primary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (1st)"

for rule in secondary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (2nd)"

df_out.columns = cols

In [54]:
df_out.sort_values('Primary Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
913,TRMT,3.992175,4.379647,5.141943,0.7056,0.2222,0.670,0.278,6.05,6.08,1.67,0.256,8.210000e+06,21099.0,False,True,True,True,True,True,True,True,True,True,True,8.0,2.0
139,RGLS,0.994380,1.155392,1.570286,0.9149,0.1837,0.881,276.778,1.84,1.96,0.44,0.391,4.782000e+07,400677.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
744,SELB,3.315512,3.673383,4.438146,0.5013,0.1254,0.496,0.791,4.57,5.07,1.64,0.529,1.079000e+08,463646.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
775,GGB,4.357226,4.529804,4.685429,0.2530,0.0514,9.513,0.429,5.17,5.42,1.72,0.299,1.700000e+09,3505978.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
686,MCF,2.458102,2.833039,4.407714,0.8148,0.1300,0.724,1.504,4.64,5.24,1.12,0.267,1.316900e+08,210184.0,False,True,True,False,True,True,True,True,True,True,True,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,EVOK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
1262,RVPH,9.002562,8.660075,8.532429,-0.1873,-0.0619,-2.787,0.000,7.62,11.50,7.49,0.039,9.230000e+06,404.0,False,False,False,True,False,False,False,False,False,False,False,0.0,1.0
748,FLNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
818,SMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0


In [55]:
df_out[df_out["Primary Passed Tests"] >= 8]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
139,RGLS,0.994380,1.155392,1.570286,0.9149,0.1837,0.881,276.778,1.84,1.9600,0.44,0.3910,4.782000e+07,400677.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
224,SGOC,1.362774,1.511078,1.954857,0.7038,0.1474,0.658,7.333,2.25,2.6000,0.80,0.0010,9.533000e+07,23325.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
267,MBII,1.543285,1.663922,2.417714,0.6437,0.1036,0.713,0.257,2.43,2.7900,0.63,0.7580,1.502300e+08,29939.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
378,XIN,2.310365,2.429412,2.771143,0.3253,0.1386,4.639,0.317,2.97,3.5300,1.70,0.0370,6.560000e+07,28344.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
542,MEIP,3.149015,3.187304,3.811000,0.1675,0.0493,0.604,8.200,3.84,4.5000,1.18,0.6940,1.125200e+08,145805.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
576,RCON,1.632555,1.869706,2.747714,1.6395,0.6222,0.502,1.376,3.80,4.2400,0.79,0.0260,4.800000e+06,125342.0,False,True,True,True,False,True,True,True,True,True,True,8.0,1.0
686,MCF,2.458102,2.833039,4.407714,0.8148,0.1300,0.724,1.504,4.64,5.2400,1.12,0.2670,1.316900e+08,210184.0,False,True,True,False,True,True,True,True,True,True,True,8.0,1.0
744,SELB,3.315512,3.673383,4.438146,0.5013,0.1254,0.496,0.791,4.57,5.0700,1.64,0.5290,1.079000e+08,463646.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
751,BKTI,3.419708,3.542255,4.357429,0.4214,0.1800,1.811,0.432,4.60,5.4200,1.17,0.6460,1.266000e+07,3241.0,False,True,True,True,False,True,True,True,True,True,True,8.0,1.0
775,GGB,4.357226,4.529804,4.685429,0.2530,0.0514,9.513,0.429,5.17,5.4200,1.72,0.2990,1.700000e+09,3505978.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
